In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
from datetime import datetime as dt
import pandas as pd

websiteCinemas='https://cinemas.com.ni/cartelera/'

movies_=[]
current_date=dt.now()
name_sales={'CNM01':'GALERÍAS','CNM02':'PLAZA INTER','CNM03':'BELLO HORIZONTE','CNM04':'VIP CAMINO DE ORIENTE','CNM05':'MASAYA'}
dataframe=pd.DataFrame(columns=['Title','Theatre Name','Country','Circuit','Showtimes'])
response_page=requests.get(websiteCinemas)
page=BeautifulSoup(response_page.text,'html.parser')


In [2]:
def sales(sale,movie_title):
    sales_dic=json.loads(sale[2:(len(sale)-2)])
    for key_sales, value in sales_dic.items():
        if len(value["Tandas"]) > 0:
            if 'fecha' in value["Tandas"][0]:
                movie_={}
                if str(current_date.day)== re.sub(r"[^0-9]",'',value["Tandas"][0]["fecha"]):
                    schedules=value["Tandas"][0]["Horarios"]
                    list_times=[]
                    movie_["Title"]=movie_title
                    movie_["Theatre Name"]=name_sales[key_sales]
                    movie_['Country']='Nicaragua'
                    movie_['Circuit']='Cinemas'
                    for key_schedules, schedules_sales in schedules.items():
                        if type(schedules_sales)== list :
                            for i in schedules_sales:
                                list_times.append(i["Hora"])
                        else:
                            for key_, value_ in schedules_sales.items():
                                list_times.append(value_["Hora"])
                        movie_["Showtimes"]=list_times
                    movies_.append(movie_)

def data_movies(url):
    response_cinema=requests.get(url)
    page_movie=BeautifulSoup(response_cinema.text,'html.parser')
    text_area=page_movie.find_all(name="textarea",attrs={"style":"display:none!important;"})
    title=page_movie.find(name='div',attrs={'id':'info-right'}).find('h2').text
    sales(str(text_area[0].contents), title)

cartelera=page.find(name="section",attrs={'id':'section-cartelera'}).find_all('a',class_="btn-red")
for movie in cartelera:
    data_movies(movie.get("href"))

In [3]:
for movie in movies_:
    df_new=pd.DataFrame(movie)
    dataframe=pd.concat([dataframe,df_new],ignore_index=True)

dataframe.to_csv('Cinemas-Nicaragua'+ '-'+str(current_date.strftime("%d-%b-%Y"))+'.csv', sep=';', encoding='utf-8', index=False)